In [1]:
from huggingface_hub import notebook_login
from datasets import load_dataset
notebook_login()

In [8]:
dataset = load_dataset("ai4bharat/samanantar","kn",data_dir="./translation_datasets/")

/home/adithya/miniconda3/envs/train-venv/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for ai4bharat/samanantar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ai4bharat/samanantar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
dataset.save_to_disk('translation_dataset')

Saving the dataset (0/2 shards):   0%|          | 0/4093524 [00:00<?, ? examples/s]

In [29]:
from datasets import load_from_disk
dataset = load_from_disk("./translation_dataset")

In [30]:
small_dataset = dataset['train'].select([i for i in range(1000000)])
small_dataset

Dataset({
    features: ['idx', 'src', 'tgt'],
    num_rows: 1000000
})

In [31]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import random

# Select a small sample (e.g., 50 rows) for testing
sampled_data = small_dataset.shuffle(seed=42).select([i for i in range(400000)])
sampled_data

Dataset({
    features: ['idx', 'src', 'tgt'],
    num_rows: 400000
})

In [33]:


# Split into two datasets
dataset1, dataset2 = train_test_split(sampled_data, test_size=0.5, random_state=42)

# Function to add a random phrase to each row with a random position
def add_random_phrase(data, source_lang, target_lang):
    phrases = [
        "translate the following to " + target_lang,
        "convert to " + target_lang,
        "translate to " + target_lang,
        "translate this to " + target_lang,
        "what is the translation of this in " + target_lang,
        " ",
        " ",
        " ",
        " ",
    ]

    data['text'] = data.apply(lambda row: f"{row['src']} {random.choice(phrases)} : {row['tgt']}" if random.choice([True, False]) else f"{random.choice(phrases)} {row['src']} : {row['tgt']}", axis=1)
    return data

# Create the first dataset with the same src and tgt
df_same = add_random_phrase(pd.DataFrame({'src': dataset1['src'], 'tgt': dataset1['tgt']}), 'english', 'kannada')

# Create the second dataset with inverted columns
df_inverted = add_random_phrase(pd.DataFrame({'src': dataset2['tgt'], 'tgt': dataset2['src']}), 'kannada', 'english')

# Save the datasets to CSV files or perform further processing as needed
df_same.to_csv('./translation_dataset/dataset_same_test.csv', index=False)
df_inverted.to_csv('./translation_dataset/dataset_inverted_test.csv', index=False)


In [34]:
# Display a sample of the datasets
print("Sample of the dataset with the same src and tgt:")
display(df_same.sample(5))

print("\nSample of the dataset with inverted columns:")
display(df_inverted.sample(5))

Sample of the dataset with the same src and tgt:


,src,tgt,text
193232,Lemon juice 1 tsp,ಶುಂಠಿ ರಸ 1 ಟೇಬಲ್ ಚಮಚ,Lemon juice 1 tsp : ಶುಂಠಿ ರಸ 1 ಟೇಬಲ್ ಚಮಚ
188555,He has been admitted to a hospital in Manipal.,ಇವರನ್ನು ಉಡುಪಿಯ ಮಣಿಪಾಲ್ ಆಸ್ಪತ್ರೆಗೆ ದಾಖಲು ಮಾಡಲಾಗ...,He has been admitted to a hospital in Manipa...
5838,Actuarial science,ಅಂಟಾರ್ಕ್ಟಿಕ್ ವಿಜ್ಞಾನ,translate the following to kannada Actuarial s...
125090,"""""""He is a businessman.""",`ಅವರು (ಸಚಿವರು) ವ್ಯಾಪಾರಿ.,"""""""He is a businessman."" convert to kannada : ..."
181508,We didnt ask anyone to resign.,ಯಾರ ರಾಜೀನಾಮೆಯನ್ನೂ ನಾವು ಕೇಳುವುದಿಲ್ಲ.,We didnt ask anyone to resign. : ಯಾರ ರಾಜೀನಾಮ...



Sample of the dataset with inverted columns:


,src,tgt,text
14516,"ಮಕ್ಕಳು ನಮ್ಮ ಸಮಾಜದ ಆಸ್ತಿ, ದೇಶದ ಮುಂದಿನ ಪ್ರಜೆಗಳು.",Children are future citizens of our society.,"ಮಕ್ಕಳು ನಮ್ಮ ಸಮಾಜದ ಆಸ್ತಿ, ದೇಶದ ಮುಂದಿನ ಪ್ರಜೆಗಳ..."
83418,""""""" ಇದು ಎರಡು ರೀತಿಯಲ್ಲಿ ಕೊಡಬಹುದು.""",This could have happened in two ways.,""""""" ಇದು ಎರಡು ರೀತಿಯಲ್ಲಿ ಕೊಡಬಹುದು."" translate ..."
43369,ಹುದ್ದೆ ಮತ್ತು ಅರ್ಹತೆ ವಿವರ,Jobs and Qualification,ಹುದ್ದೆ ಮತ್ತು ಅರ್ಹತೆ ವಿವರ : Jobs and Qualific...
67542,ಆದರೆ “ಯಾವಾಗಲೂ ನಗುನಗುತ್ತಾ ತುಂಬ ಸಂತೋಷವಾಗಿದ್ದ ಕುಟ...,"No one ever saw a trailer like it before, reca...",ಆದರೆ “ಯಾವಾಗಲೂ ನಗುನಗುತ್ತಾ ತುಂಬ ಸಂತೋಷವಾಗಿದ್ದ ಕುಟ...
160483,ನನಗೆ ನಡೆದಾಡಲು ಸಾಧ್ಯವಾಗುತ್ತಿಲ್ಲ.,I cannot walk.,translate to english ನನಗೆ ನಡೆದಾಡಲು ಸಾಧ್ಯವಾಗುತ್...


In [35]:
from datasets import Dataset

# Convert pandas DataFrames to datasets
dataset_same = Dataset.from_pandas(df_same)
dataset_inverted = Dataset.from_pandas(df_inverted)

# # Save datasets to disk
# dataset_same.save_to_disk('./translation_datasets/dataset_same')
# dataset_inverted.save_to_disk('./translation_datasets/dataset_inverted')
print(dataset_same)
print(dataset_inverted)

Dataset({
    features: ['src', 'tgt', 'text'],
    num_rows: 200000
})
Dataset({
    features: ['src', 'tgt', 'text'],
    num_rows: 200000
})


In [36]:
# Upload datasets to Hugging Face Hub (requires authentication)
# Make sure to replace 'username' and 'dataset-name' with your Hugging Face username and desired dataset name
dataset_same.push_to_hub('CognitiveLab/English_to_Kannada' ,private=True)
dataset_inverted.push_to_hub('CognitiveLab/Kannada_to_English',private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/200 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/200 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/CognitiveLab/Kannada_to_English/commit/dd23e0cda3bc694ce0202e36d355c9f44123a48b', commit_message='Upload dataset', commit_description='', oid='dd23e0cda3bc694ce0202e36d355c9f44123a48b', pr_url=None, pr_revision=None, pr_num=None)

In [37]:
# df_same = pd.read_csv('./translation_datasets/dataset_same_test.csv')
# df_inverted = pd.read_csv('./translation_datasets/dataset_inverted_test.csv')

# Add extra field for "en_to_kn" or "kn_to_en"
df_same['translation_direction'] = 'en_to_kn'
df_inverted['translation_direction'] = 'kn_to_en'

# Combine both datasets
df_same

,src,tgt,text,translation_direction
0,"Abdul Jaleel K., the district president of the...",ಕಾರ್ಯಕ್ರಮದ ಅಧ್ಯಕ್ಷತೆಯನ್ನು ಎಸ್ ಡಿ ಪಿ ಐ ಸುಳ್ಯ ಕ್...,"Abdul Jaleel K., the district president of the...",en_to_kn
1,"""Write this down.""""""",ಅದನ್ನು ಬರೆದುಕೊಳ್ಳಿ' ಎಂದು ನಿಜಕ್ಕೂ ಸತ್ತುಹೋದ.,"""Write this down."""""" translate to kannada : ಅದ...",en_to_kn
2,RTI office,ಆರ್ ಟಿಓ ಕಚೇರಿ,RTI office : ಆರ್ ಟಿಓ ಕಚೇರಿ,en_to_kn
3,"Sa in Sanskrit is to have, and Misha in Russia...","ಸ ಎಂದರೆ ಸಂಸ್ಕೃತದಲ್ಲಿ ಹೊಂದುವುದು ಎಂದರ್ಥ, ಹಾಗೆಯೇ ...",translate this to kannada Sa in Sanskrit is to...,en_to_kn
4,Thousands of trucks carrying essential commodi...,ಕರ್ನಾಟಕ ಮತ್ತು ಮಹಾರಾಷ್ಟ್ರಕ್ಕೆ ಸಂಪರ್ಕ ಕಲ್ಪಿಸುವ ಈ...,translate the following to kannada Thousands o...,en_to_kn
...,...,...,...,...
199995,The mind is fickle.,ಮನಸ್ಸು ಚುರುಕ್ ಅಂತು.,translate the following to kannada The mind is...,en_to_kn
199996,"""""""Hey pops.""","""""""ಮೇರಿ ಪಾಪಿನ್ಸ್.""","convert to kannada """"""Hey pops."" : """"""ಮೇರಿ ಪಾ...",en_to_kn
199997,He was earlier associated with Aam Aadmi Party.,ಅವರು ಈ ಹಿಂದೆ ಅಕ್ ಪಾರ್ಟಿಯ ಮೈತ್ರಿಯಲ್ಲಿದ್ದರು.,He was earlier associated with Aam Aadmi Party...,en_to_kn
199998,The film was directed by Raj.,ಚಿತ್ರವನ್ನು ರಾಜ್ ಬಿಎನ್ ನಿರ್ದೇಶಿಸಿದ್ದಾರೆ.,The film was directed by Raj. translate the fo...,en_to_kn


In [38]:
combined_df = pd.concat([df_same, df_inverted], ignore_index=True)

# Shuffle the combined dataset randomly
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert the combined DataFrame to a dataset
combined_dataset = Dataset.from_pandas(combined_df)

combined_dataset

Dataset({
    features: ['src', 'tgt', 'text', 'translation_direction'],
    num_rows: 400000
})

In [39]:
# Save the combined dataset to disk
combined_dataset.save_to_disk('./translation_datasets/combined_dataset')

# Upload the combined dataset to Hugging Face Hub (requires authentication)
# Make sure to replace 'username' and 'dataset-name' with your Hugging Face username and desired dataset name
combined_dataset.push_to_hub('CognitiveLab/translation-combined-dataset')

Saving the dataset (0/1 shards):   0%|          | 0/400000 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/400 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/CognitiveLab/translation-combined-dataset/commit/f14c910e390827fc217c783c9cee01e96f21adc7', commit_message='Upload dataset', commit_description='', oid='f14c910e390827fc217c783c9cee01e96f21adc7', pr_url=None, pr_revision=None, pr_num=None)